In [ ]:
!conda install -c conda-forge gdcm -y
!cp ../input/gdcm-data/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
print("done")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
from pydicom import dcmread # for dcm files
import pydicom
import cv2
import csv

from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py
import gdcm
import PIL
import tensorflow as tf
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# LIST OF PATHS

In [ ]:
"""
path = '../input/rsna-str-pulmonary-embolism-detection/test'
image_paths_test = []
SOPs_test = []
series_test = []
studies_test = []
studies_paths_test = []
series_paths_test = []
import os
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        temp = os.path.join(dirname, filename)
        
        temp_series_path = temp[0:temp.rfind('/')]
        series_paths_test.append(temp_series_path)
        
        temp_series = temp_series_path[temp_series_path.rfind('/')+1:len(temp_series_path)]
        series_test.append(temp_series)
        
        temp_study_path = temp_series_path[0:temp_series_path.rfind('/')]
        studies_paths_test.append(temp_study_path)
        
        temp_study = temp_study_path[temp_study_path.rfind('/')+1:len(temp_study_path)]
        studies_test.append(temp_study)
        
        image_paths_test.append(os.path.join(dirname, filename))
        SOPs_test.append(temp[len(temp)-16:len(temp)-4])
        
        

image_paths_test = np.asarray(image_paths_test)
SOPs_test = np.asarray(SOPs_test)
series_test = np.asarray(series_test)
studies_test = np.asarray(studies_test)
studies_paths_test = np.asarray(studies_paths_test)
series_paths_test = np.asarray(series_paths_test)
"""

In [ ]:
"""
image_paths_test = []
SOPs_test = []
series_test = []
studies_test = []
studies_paths_test = []
series_paths_test = []
import os
for dirname, _, filenames in os.walk('../input/rsna-str-pulmonary-embolism-detection/test'):
    for filename in filenames:
        temp = os.path.join(dirname, filename)
        
        temp_series_path = temp[0:temp.rfind('/')]
        series_paths_test.append(temp_series_path)
        
        temp_series = temp_series_path[temp_series_path.rfind('/')+1:len(temp_series_path)]
        series_test.append(temp_series)
        
        temp_study_path = temp_series_path[0:temp_series_path.rfind('/')]
        studies_paths_test.append(temp_study_path)
        
        temp_study = temp_study_path[temp_study_path.rfind('/')+1:len(temp_study_path)]
        studies_test.append(temp_study)
        
        image_paths_test.append(os.path.join(dirname, filename))
        SOPs_test.append(temp[len(temp)-16:len(temp)-4])
        
        

image_paths_test = np.asarray(image_paths_test)
SOPs_test = np.asarray(SOPs_test)
series_test = np.asarray(series_test)
studies_test = np.asarray(studies_test)
studies_paths_test = np.asarray(studies_paths_test)
series_paths_test = np.asarray(series_paths_test)
"""

In [ ]:
"""
print(temp)
temp_series_path = temp[0:temp.rfind('/')]
temp_series = temp_series_path[temp_series_path.rfind('/')+1:len(temp_series_path)]
temp_study_path = temp_series_path[0:temp_series_path.rfind('/')]
temp_study = temp_study_path[temp_study_path.rfind('/')+1:len(temp_study_path)]

print(temp_study_path)
print("study",temp_study)

print(temp_series_path)
print("series",temp_series)
"""

In [ ]:
with open ('../input/rsna-str-pulmonary-embolism-detection/test.csv') as csvfile:  
    readCSVFeatures = csv.reader(csvfile, delimiter=',')
    test_data = list(csv.reader(csvfile))
test_data = test_data[1:len(test_data )]    
test_data = np.asarray(test_data)

In [ ]:
len(test_data)

In [ ]:
study_test_csv = test_data[:,0]
test_path = np.array(['../input/rsna-str-pulmonary-embolism-detection/test/' for _ in range(len(test_data[:,0]))])
slash_array = np.array(['/' for _ in range(len(test_data[:,0]))])
dcm_array = np.array(['.dcm' for _ in range(len(test_data[:,0]))])
study_path = np.char.add(test_path,test_data[:,0]) 
studies_test = test_data[:,0]
temp_array = np.char.add(study_path,slash_array) 
series_path = np.char.add(temp_array,test_data[:,1]) 
series_path = np.char.add(series_path,slash_array) 
temp_array = np.char.add(series_path,slash_array) 
sops_path = np.char.add(temp_array,test_data[:,2]) 
SOPs_test = test_data[:,2]
sops_path = np.char.add(sops_path,dcm_array) 
study_path_temp,study_path_unique_indexes =  np.unique(studies_test,return_index = True)
sorted_path  =np.sort(study_path_unique_indexes)
study_path_unique = study_path[sorted_path]
studies_test_unique = studies_test[sorted_path]


In [ ]:
print(studies_test[-1])
print(sops_path[-1])
print(test_data[:,0][-1])
print(len(sops_path))

In [ ]:
"""
print(len(test_data))
print(len(image_paths_test))
"""

In [ ]:
"""
print(image_paths_test[0])
print(studies_test[0])
studies_test_index= np.argsort(studies_test)
image_paths_test= image_paths_test[studies_test_index]
studies_test= studies_test[studies_test_index]
SOPs_test = SOPs_test[studies_test_index]
series_test = series_test[studies_test_index]
studies_paths_test = studies_paths_test[studies_test_index] 
series_paths_test = series_paths_test[studies_test_index]

print(image_paths_test[0])
print(studies_test[0])

study_test_csv = test_data[:,0]
study_test_csv_index= np.argsort(study_test_csv)
test_data = test_data[study_test_csv_index]
study_test_csv = study_test_csv[study_test_csv_index]

print(image_paths_test[0])
print(test_data[0])
print(studies_test[0])
print(studies_paths_test[0])
study_test_csv_unique,study_test_csv_uniq_indexes =  np.unique(study_test_csv,return_index = True)
"""

In [ ]:

def preprocess_image(path):
    dataset = dcmread(path)
    image_dene = np.asarray(dataset.pixel_array)
    preprocessed_image= cv2.resize(image_dene , (150, 150),  interpolation = cv2.INTER_AREA)
    preprocessed_image = cv2.merge((preprocessed_image,preprocessed_image,preprocessed_image))
    preprocessed_image = tf.keras.applications.xception.preprocess_input(preprocessed_image) 

    return preprocessed_image
    
    
      
    


In [ ]:
model = keras.models.load_model('../input/model-last-second-epoch/model_last_second_epoch.h5')

In [ ]:

count_no = 80

#image_paths_test = image_paths_test[0:study_test_csv_uniq_indexes[1700]]
len_dataset = len(sops_path)

pred_all = np.asarray([])
for count in range(0,count_no):
    print(count)
    start = int((len_dataset/count_no)*count)
    end= int((len_dataset/count_no)*(count+1))
    x_test = np.ones((end-start, 150, 150, 3),  dtype=np.int16)
    print("start",start)
    print("end",end)
    for i in range(start,end):
        if i % 1000 == 0 :
            print(i) 
        x_test[i-start]= preprocess_image(sops_path[i])
    #pred = np.ones((end-start),  dtype=np.int16)
    pred = model.predict(x_test)
    pred_binary = np.ones(len(x_test))
    
    print(len(np.where(pred>0.5)[0]))
    print(len(np.where(pred<=0.5)[0]))
    
    pred_binary[np.where(pred>0.5)[0]] = 1
    pred_binary[np.where(pred<=0.5)[0]] = 0    
    
    #pred_all = np.concatenate((pred_all,pred_binary),axis = None)
    pred_all = np.concatenate((pred_all,pred),axis = None)


In [ ]:
"""
count_no = 100

image_paths_test = image_paths_test[0:study_test_csv_uniq_indexes[1700]]
len_dataset = len(image_paths_test)
print(len_dataset)
pred_all = []
for i in range(0,len_dataset):
    if i %100== 0:
        print(i)
    x_test= preprocess_image(image_paths_test[i])
    #print(x_test.shape)
    x_test = np.reshape(x_test,(1,150,150,3))
    pred = model.predict(x_test)
    if pred>0.5:
        pred_binary = 1
    if pred<=0.5:
        pred_binary = 0    
 
    
    pred_all.append(pred_binary)

pred_all = np.asarray(pred_all)
"""

In [ ]:
x_test=[]

In [ ]:
#print(len(pred_all))
#print(len(SOPs_test))

In [ ]:

"""

x_test = np.ones((int(len(image_paths_test)/10), 150, 150, 3),  dtype=np.int16)
for i in range(0,int(len(image_paths_test)/10)):
    if i % 1000 == 0 :
        print(i) 
    x_test[i]= preprocess_image(image_paths_test[i])

pred = model.predict(x_test)
pred_binary = np.ones(len(x_test))
print(len(np.where(pred>0.5)[0]))
print(len(np.where(pred<=0.5)[0]))

pred_binary[np.where(pred>0.5)[0]] = 1
pred_binary[np.where(pred<=0.5)[0]] = 0  
"""

In [ ]:
"""
negative_exam = np.zeros(len(study_test_csv_unique))
rv_lv_ratio_lt_1 = np.zeros(len(study_test_csv_unique))
rv_lv_ratio_gte_1 = np.zeros(len(study_test_csv_unique))
leftsided_pe = np.zeros(len(study_test_csv_unique))
chronic_pe = np.zeros(len(study_test_csv_unique))
rightsided_pe = np.zeros(len(study_test_csv_unique))
acute_and_chronic_pe = np.zeros(len(study_test_csv_unique))
central_pe = np.zeros(len(study_test_csv_unique))
indeterminate = np.zeros(len(study_test_csv_unique))

for x in range(len(study_test_csv_unique)):       
    del_index = study_test_csv_uniq_indexes[x]
    if x < len(study_test_csv_unique)-1:
        del_index_2 = study_test_csv_uniq_indexes[x+1]   
    else:  
        del_index_2 = len(SOPs_test)
    negative = True
    for i in range(del_index,del_index_2):
          
        if pred_all[i] == 1:
            negative = False
    
    if negative = True:
        negative_exam[x] = 1
    
    if negative = False:
        negative_exam[x] = 0       
"""

In [ ]:
SOPs_test

In [ ]:

#del_index = studies_test_unique[0]
#del_index_2 = studies_test_unique[1]

import csv

with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file) 
    writer.writerow(["id", "label"])
    for x in range(len(studies_test_unique)):       
        writer.writerow([str(studies_test_unique[x])+"_negative_exam_for_pe", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_rv_lv_ratio_gte_1", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_rv_lv_ratio_lt_1", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_leftsided_pe", str(0.5)])    
        writer.writerow([str(studies_test_unique[x])+"_chronic_pe", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_rightsided_pe", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_acute_and_chronic_pe", str(0.5)])
        writer.writerow([str(studies_test_unique[x])+"_central_pe", str(0.5)])   
        writer.writerow([str(studies_test_unique[x])+"_indeterminate", str(0.5)]) 
    

    for i in range(0,len(SOPs_test)):
        writer.writerow([str(SOPs_test[i]), str(pred_all[i])]) 



In [ ]:
with open ('./submission.csv') as csvfile:  
    readCSVFeatures = csv.reader(csvfile, delimiter=',')
    submission_data = list(csv.reader(csvfile))

submission_data = submission_data[1:len(submission_data)]    
submission_data = np.asarray(submission_data)
print(len(submission_data))


In [ ]:
print(9*len(studies_test_unique)+len(sops_path))